In [1]:
# 2 weeks together
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2) #+7

write_folder="/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) 

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]
recent_2_weeks

[datetime.date(2020, 8, 22), datetime.date(2020, 8, 15)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_2_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_1_after_201806=list_1_after_201806+list_2_after_201806
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


In [4]:
new_2_biweekly_files

['/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-08-15/MediaStormMasterWeekly20200818-113056-724.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-08-22/MediaStormMasterWeekly20200825-113108-479.txt']

In [5]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
        print(df_all_new_master.shape)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))


(167771, 3)
(339167, 3)
339167
339151


In [6]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

273


In [7]:
previous_id_files_lists

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_20

In [8]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/BiWeekly_New_SignUps/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]

uploaded_new_seg.sort()
uploaded_new_seg=uploaded_new_seg[-24:]

print("len(uploaded_new_seg)"+str(len(uploaded_new_seg)))
previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

len(uploaded_new_seg)24


In [9]:
# Exclude the quarterly regular segments


In [10]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index),file_uploaded)

2020-08-26 10:18:08.691465 339151 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:18:09.385665 339151 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-08-26 10:18:09.816721 339151 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:18:10.018431 339151 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-08-26 10:18:30.854636 339149 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_

2020-08-26 10:19:59.614798 339137 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:20:00.033313 339137 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-08-26 10:20:00.145432 339137 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:20:00.268406 339136 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-08-26 10:20:00.373002 339136 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_P

2020-08-26 10:20:31.779637 339135 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-08-26 10:20:32.631921 339135 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:20:33.207850 339135 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-08-26 10:20:33.518938 339135 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:20:33.991125 339135 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_P_Above50K_Mu

2020-08-26 10:21:47.632910 339127 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:21:48.099199 339127 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-08-26 10:21:50.500897 339127 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:21:50.758227 339127 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-08-26 10:22:10.633515 339126 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/C_all_quarterly_segments_RFM20Q1.csv
2020-0

2020-08-26 10:22:40.384668 339124 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:22:40.767292 339124 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-08-26 10:22:41.139209 339124 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:22:41.251100 339124 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-08-26 10:22:56.622303 339121 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_S

2020-08-26 10:23:31.632503 339110 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:23:31.853224 339110 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-08-26 10:23:33.418080 339110 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:23:33.537143 339110 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-08-26 10:23:33.646128 339109 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_L_P_Below50K_

2020-08-26 10:24:05.043160 339105 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:24:05.260867 339105 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-08-26 10:24:07.074935 339104 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:24:07.235470 339104 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-08-26 10:24:07.351640 339104 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Active_M_S_Above50K_MultiFamilyOrCondo_Compe

2020-08-26 10:24:26.490616 339101 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Lapsed1318_L_S_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-08-26 10:24:29.717871 339101 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-05-02/NewReward_2020-05-02_P.csv
2020-08-26 10:24:29.988326 339101 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-05-02/NewReward_2020-05-02_S.csv
2020-08-26 10:24:39.957893 339099 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-05-02/NewReward_2020-05-02_T.csv
2020-08-26 10:24:44.144652 339099 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-05-16/NewReward_2020-05-16_P.csv
2020-08-26 10:24:44.378725 339099 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-05-16/NewReward_2020-05-16_S.csv
2020-08-26 10:24:46.607007 339099 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-05-16/NewReward_2020

In [11]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

339098


In [12]:
zipmap = pd.ExcelFile('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx')
zipmap=zipmap.parse("unique_zips_full_footprint",dtype=str)
zipmap['zip_type']=zipmap['zip_type'].replace("zips_10","trans_S")

zipmap['zipcodegroup'] = zipmap['zip_type'].apply(lambda x: x.split("_")[1])
print(zipmap.shape)
print(zipmap['zip_cd'].nunique())

zipmap=zipmap.rename(columns={"zip_cd":"customer_zip_code"})

zipmap=zipmap[['customer_zip_code','zipcodegroup']]
zipmap.head(2)

(12980, 3)
12980


,customer_zip_code,zipcodegroup
0,75040,P
1,20743,P


In [13]:
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())

df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment_new_signups'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots /uploads/big_lots_onboarding

['P' nan 'S']
['P' 'T' 'S']
P 120102
S 49506
T 169490


In [14]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,120102
1,S,49506
2,T,169490


In [15]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment_new_signups
1,f90f79908132b6dc34198756fc6bdcaabf3478ccc669a5...,9252b51047b85611415558141a31c4b38049d953183be0...,NaN,NewReward_082220_T
2,f4028968ce1ed20105691ece10cf5d585d99c2021a6fa6...,536c5cbec5722fc868868eb5ab9cffeb488edd8d75244b...,NaN,NewReward_082220_T


In [16]:
df_all_new_master.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,zipcodegroup
0,968894452a329dd4bed0f823d496afffc1cdf5e4524ccc...,8d1f1e788a1684e195253d17bc5e0195ff0107935efa55...,78223,P
1,f90f79908132b6dc34198756fc6bdcaabf3478ccc669a5...,9252b51047b85611415558141a31c4b38049d953183be0...,NaN,T


In [17]:
df_all_new_master[pd.isnull(df_all_new_master['customer_zip_code'])].shape

(147897, 4)

In [18]:
# to do below 
print(datetime.datetime.now())

2020-08-26 10:28:54.563087


In [19]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
transport.close()
print("Done: ",datetime.datetime.now())

Done:  2020-08-26 10:29:10.317525


In [22]:
os.getcwd()

'/home/jian/celery/BiWeekly_New_SignUps'